In [ ]:
import numpy as np
import tensorflow as tf

#Data
x_train = np.array([[1,2,1,1], [2,1,3,2], [3,1,3,4], [4,1,5,5], [1,7,5,5], [1,2,5,6],
                   [1,6,6,6], [1,7,7,7]], dtype = np.float32)
y_train = np.array([[0,0,1], [0,0,1], [0,0,1], [0,1,0], [0,1,0], [0,1,0], 
                   [1,0,0], [1,0,0]], dtype = np.float32)
W = tf.Variable(tf.random.normal([4,3]))
b = tf.Variable(tf.random.normal([3]))

#H(x) 값을 출력함. x_data 행렬과 W행렬을 곱해서 하나의 값을 뽑아냄
def model_SoftmaxClassificationLC(x):
  return tf.matmul(x, W) + b

# (데이터 - 정답)을 제곱(square)하고, 평균값내기(reduce_mean)
def cost_SoftmaxClassificationLC(model_x) :
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model_x,
                                                                labels = y_train))

# SGD: 경사하향법 하는 함수, 인자(learning rate)는 gradient양
def train_optimization(x) :
  with tf.GradientTape() as g:
    model_LC = model_SoftmaxClassificationLC(x) #x의 3*2와 랜덤(w) 2*1 행렬 곱해서, 3*1짜리 생성
    cost = cost_SoftmaxClassificationLC(model_LC)
  gradients = g.gradient(cost, [W, b]) #나온 오차(cost)에 [W,b]를 보내, 미분+기울기 구함

    #오차역전파 구해, weight을 재정의함(SGD - Learning rate)
    #zip(gradinets, [W,b])를 통해, 다시 정의할 필요 없이 바로 다시 둘을 사용 가능
  tf.optimizers.SGD(0.01).apply_gradients(zip(gradients, [W,b]))

In [ ]:
for step in range(2001) :
  train_optimization(x_train)
  if step%100 == 0:
    pred = model_SoftmaxClassificationLC(x_train)
    loss = cost_SoftmaxClassificationLC(pred)
    correct = tf.equal(tf.argmax(pred, 1), tf.argmax(y_train, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    print("Step: {}, \t Accuracy: {}, \t Loss:{}".format(step, accuracy.numpy().flatten(),
                                                         loss))

print('='*30)
x_test = np.array([[1,8,8,8]], dtype = np.float32)
model_test = tf.nn.softmax(model_SoftmaxClassificationLC(x_test))
print('Model Ttest')
print("model for [1,8,8,8]: ", tf.argmax(model_test, 1).numpy())
print('=' * 30)

Step: 0, 	 Accuracy: [0.375], 	 Loss:13.390893936157227
Step: 100, 	 Accuracy: [0.375], 	 Loss:1.115812063217163
Step: 200, 	 Accuracy: [0.375], 	 Loss:0.9250802993774414
Step: 300, 	 Accuracy: [0.5], 	 Loss:0.7996666431427002
Step: 400, 	 Accuracy: [0.625], 	 Loss:0.719741940498352
Step: 500, 	 Accuracy: [0.875], 	 Loss:0.6675014495849609
Step: 600, 	 Accuracy: [0.875], 	 Loss:0.6312598586082458
Step: 700, 	 Accuracy: [0.875], 	 Loss:0.6044679880142212
Step: 800, 	 Accuracy: [0.875], 	 Loss:0.5835670232772827
Step: 900, 	 Accuracy: [0.875], 	 Loss:0.5665581822395325
Step: 1000, 	 Accuracy: [0.875], 	 Loss:0.5522572994232178
Step: 1100, 	 Accuracy: [0.875], 	 Loss:0.539922833442688
Step: 1200, 	 Accuracy: [0.875], 	 Loss:0.5290670990943909
Step: 1300, 	 Accuracy: [0.875], 	 Loss:0.5193561911582947
Step: 1400, 	 Accuracy: [0.875], 	 Loss:0.5105538368225098
Step: 1500, 	 Accuracy: [0.875], 	 Loss:0.5024868249893188
Step: 1600, 	 Accuracy: [0.875], 	 Loss:0.4950265884399414
Step: 1700, 	 

In [ ]:
import tensorflow as tf
import numpy as np
import torch.nn as nn
from tensorflow.keras.datasets import mnist

batch_size = 128 #60000개를 한번에 행렬로 가져올 수 없으니
#128개씩 나눠가져옴

#사용할 중간 레이어들. 256으로 되어있다.
#256으로 되어있을땐 확률이 91%로 나온다.

#만일 히든레이어를 돌리지 않을 경우, 확률은 86%로 떨어지는 것을 알 수 있다.(softmax)
#또한, 히든레이어 노드개수를 1로 설정할 경우엔, 10%대로 대폭 감소한다.
nH1=256
nH2=256
nH3=256

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
#데이터를 [총개수/784, 784]로 변경 
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
x_train, x_test = x_train/255., x_test/255. #0~1사이로 변경
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

#numpy array나 list를 tensor dataset으로 변환
#이후, 데이터셋을 임의로 배치사이즈만큼 섞음
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size)

#히든레이어 적용
#W는 가중치넣는 배열, b는 bias개수
class ANN(nn.Module) :
  def __init__(self):
    self.W_1 = tf.Variable(tf.random.normal(shape=[784, nH1]))
    self.W_2 = tf.Variable(tf.random.normal(shape=[nH1, nH2]))
    self.W_3 = tf.Variable(tf.random.normal(shape=[nH2, nH3]))
    self.W_Out = tf.Variable(tf.random.normal(shape=[nH3, 10]))
    #self.W_Out = tf.Variable(tf.random.normal(shape=[784, 10]))
    self.b_1 = tf.Variable(tf.random.normal(shape=[nH1]))
    self.b_2 = tf.Variable(tf.random.normal(shape=[nH1]))
    self.b_3 = tf.Variable(tf.random.normal(shape=[nH1]))
    self.b_Out = tf.Variable(tf.random.normal(shape=[10]))
    #self.b_Out = tf.Variable(tf.random.normal(shape=[10]))

  def __call__(self, x) :
    H1_Out = tf.nn.relu(tf.matmul(x, self.W_1) + self.b_1)
    H2_Out = tf.nn.relu(tf.matmul(H1_Out, self.W_2) + self.b_2)
    H3_Out = tf.nn.relu(tf.matmul(H2_Out, self.W_3) + self.b_3)
    Out = tf.matmul(H3_Out, self.W_Out) + self.b_Out
    #Out = tf.matmul(x, self.W_Out) + self.b_Out
    return Out

In [ ]:
W_12 = tf.Variable(tf.random.normal(shape=[784, nH1]))
W_22 = tf.Variable(tf.random.normal(shape=[nH1, nH2]))
W_32 = tf.Variable(tf.random.normal(shape=[nH2, nH3]))
W_Out2 = tf.Variable(tf.random.normal(shape=[nH3, 10]))
    #self.W_Out = tf.Variable(tf.random.normal(shape=[784, 10]))
b_12 = tf.Variable(tf.random.normal(shape=[nH1]))
b_22 = tf.Variable(tf.random.normal(shape=[nH1]))
b_32 = tf.Variable(tf.random.normal(shape=[nH1]))
b_Out2 = tf.Variable(tf.random.normal(shape=[10]))


def caller(x) :
  H1_Out = tf.nn.relu(tf.matmul(x, W_12) + b_12)
  H2_Out = tf.nn.relu(tf.matmul(H1_Out, W_22) + b_22)
  H3_Out = tf.nn.relu(tf.matmul(H2_Out, W_32) + b_32)
  Out = tf.matmul(H3_Out, W_Out2) + b_Out2
  #Out = tf.matmul(x, self.W_Out) + self.b_Out
  return Out

In [ ]:

\ANN_model = ANN() #__init__ 실행
optimizer = tf.optimizers.Adam(0.01)

#softmax 비용함수(비용 구하기) 상세한건 이전거 참조
def cost_ANN_mnist(x,y) :
  y = tf.cast(y, tf.int64)
  loss = tf.nn.softmax_cross_entropy_with_logits(logits=x, labels=y)
  return tf.reduce_mean(loss)

#tf.argmax 비교 통해, 맞는지 확인
def accuracy(x,y) :
  correct = tf.equal(tf.argmax(x,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
  return accuracy

def train_optimization(x,y) :
  with tf.GradientTape() as g:
    pred = caller(x) #Ann 모델에 x 데이터 삽입
    cost = cost_ANN_mnist(pred, y) #비용 구하기
  gradients = g.gradient(cost,[W_12,
                                             b_Out2])
  #기존 gradients와 다르게 뒤에 unconnected...를 정의함으로써, 연결되지않는 그래디언트로 인한 
  #오류를 제거
  optimizer.apply_gradients(zip(gradients, [W_12
                                            , b_Out2]))

#300개씩 가져와 batch_x, batch_y에 넣음
#섞인 train data들을, 300개(+y는 1개씩)가져와 batch_x, y에 넣음
for step, (batch_x, batch_y) in enumerate(train_data.take(300), 1):
  train_optimization(batch_x, batch_y)
  #batch_x 넣고, 러닝 돌린다. 이를 batch_y와 비교해, 손실 구하고
  #정확도 역시 판단.
  if step%10 == 0:
    pred = caller(batch_x) #__call__ 실행
    loss = cost_ANN_mnist(pred, batch_y)
    acc = accuracy(pred, batch_y)
    print("Step: {}, \t Accuracy: {}, \t Loss:{}".format(step, acc.numpy().flatten(),
                                                         loss.numpy().flatten()))

print('='*30)
print('Model Ttest')
print("Test accuracy: ", accuracy(caller(x_test), y_test).numpy().flatten())
print('=' * 30)

Step: 10, 	 Accuracy: [0.9296875], 	 Loss:[210.1358]
Step: 20, 	 Accuracy: [0.9296875], 	 Loss:[184.19086]
Step: 30, 	 Accuracy: [0.9140625], 	 Loss:[304.43677]
Step: 40, 	 Accuracy: [0.96875], 	 Loss:[36.94867]
Step: 50, 	 Accuracy: [0.9296875], 	 Loss:[228.90591]
Step: 60, 	 Accuracy: [0.9375], 	 Loss:[154.9779]
Step: 70, 	 Accuracy: [0.921875], 	 Loss:[273.5207]
Step: 80, 	 Accuracy: [0.9140625], 	 Loss:[195.20493]
Step: 90, 	 Accuracy: [0.921875], 	 Loss:[144.4783]
Step: 100, 	 Accuracy: [0.9609375], 	 Loss:[99.49069]
Step: 110, 	 Accuracy: [0.921875], 	 Loss:[210.35207]
Step: 120, 	 Accuracy: [0.953125], 	 Loss:[95.05138]
Step: 130, 	 Accuracy: [0.96875], 	 Loss:[48.155903]
Step: 140, 	 Accuracy: [0.9296875], 	 Loss:[124.88468]
Step: 150, 	 Accuracy: [0.921875], 	 Loss:[157.46167]
Step: 160, 	 Accuracy: [0.921875], 	 Loss:[158.962]
Step: 170, 	 Accuracy: [0.921875], 	 Loss:[209.60706]
Step: 180, 	 Accuracy: [0.9609375], 	 Loss:[43.99639]
Step: 190, 	 Accuracy: [0.90625], 	 Loss:[1

In [ ]:
ran1 = tf.Variable(tf.random.normal(shape = [10]))
ran2 = tf.Variable(tf.random.normal(shape = [1,10]))

ran1

<tf.Variable 'Variable:0' shape=(10,) dtype=float32, numpy=
array([-0.18430068, -1.7281617 , -0.8465648 , -0.33136225, -0.16282019,
        0.6402151 ,  0.06544773,  0.7763703 , -0.12220418,  0.16584687],
      dtype=float32)>

In [ ]:
ran2

<tf.Variable 'Variable:0' shape=(1, 10) dtype=float32, numpy=
array([[ 0.25651702,  0.3352808 , -0.524567  ,  0.12463368, -1.1703341 ,
         0.42329538,  0.13766249,  0.73083544,  0.6887514 ,  0.2929625 ]],
      dtype=float32)>